In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np


class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

# 70で作成した行列をロード
X_train = np.load("./matrix/x_train.npy")
Y_train = np.load("./matrix/y_train.npy")

X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor =torch.from_numpy(Y_train)
datasets = TextDataset(X_train, Y_train)
train_dataloader = DataLoader(datasets, shuffle=True, batch_size=64)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(300, 4),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.layer(x)
        return logits

model = Model()
learning_rate = 1e-3
batch_size = 64
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [2]:
# train
for t in range(epochs):
    size = len(datasets)
    epoch_loss = 0
    epoch_correct = 0
    total_samples = 0

    for batch, (X, y) in enumerate(train_dataloader):
        # 予測と損失の計算
        X = X.float()
        pred = model(X)
        loss = loss_fn(pred, y.to(torch.float64))

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 損失の累計
        epoch_loss += loss.item()

        # 正答数の累計
        predicted = pred.argmax(dim=1)
        correct = (predicted == y.argmax(dim=1)).sum().item()
        epoch_correct += correct
        total_samples += len(y)

    # エポックごとの平均損失と正答率を計算
    avg_loss = epoch_loss / len(train_dataloader)
    avg_accuracy = epoch_correct / total_samples

    # パラメータと最適化アルゴリズムの状態を保存
    model_state_path = f"76_model/model_state_epoch_{t+1}.pth"
    optimizer_state_path = f"76_optimizer/optimizer_state_epoch_{t+1}.pth"
    torch.save(model.state_dict(), model_state_path)
    torch.save(optimizer.state_dict(), optimizer_state_path)

    print(f"Epoch {t+1}: Average Loss: {avg_loss}, Average Accuracy: {avg_accuracy}")
    print(f"Model parameters and optimizer state saved for epoch {t+1}")


/Users/tajirimanato/.pyenv/versions/3.12.2/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch 1: Average Loss: 1.386717721256504, Average Accuracy: 0.22434635929153782
Model parameters and optimizer state saved for epoch 1
Epoch 2: Average Loss: 1.3846114933590898, Average Accuracy: 0.2524599381501265
Model parameters and optimizer state saved for epoch 2
Epoch 3: Average Loss: 1.3825066917872415, Average Accuracy: 0.28038609314965796
Model parameters and optimizer state saved for epoch 3
Epoch 4: Average Loss: 1.3803946189449454, Average Accuracy: 0.3046574828975729
Model parameters and optimizer state saved for epoch 4
Epoch 5: Average Loss: 1.378290615608501, Average Accuracy: 0.32377471652141315
Model parameters and optimizer state saved for epoch 5
Epoch 6: Average Loss: 1.3761889204567523, Average Accuracy: 0.33923718489363697
Model parameters and optimizer state saved for epoch 6
Epoch 7: Average Loss: 1.3740877174790238, Average Accuracy: 0.3558241964202043
Model parameters and optimizer state saved for epoch 7
Epoch 8: Average Loss: 1.371985485700845, Average Acc